In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("data/train (7).csv", index_col=0)
train_data = df.copy()

In [3]:
test = pd.read_csv('data/test (4).csv', index_col=0)

In [4]:
X = df.drop(columns=['target'])
y = df['target']

In [6]:
from sklearn.feature_selection import RFE
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
n_folds = 15
weights = []
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
train_data = X
train_target = y

estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=92, step=1)
selector.fit(train_data, train_target)
rfe_train_data = train_data.iloc[:, selector.support_]
test_data = test.iloc[:, selector.support_]
test_preds = np.zeros((test_data.shape[0],))

for fold, (train_idx, val_idx) in enumerate(kf.split(train_data)):
    X_train, y_train = rfe_train_data.iloc[train_idx], train_target[train_idx]
    X_val, y_val = rfe_train_data.iloc[val_idx], train_target[val_idx]

    model = LinearRegression()
    model.fit(X_train, y_train)

    val_preds = model.predict(X_val)
    fold_rmse = mean_squared_error(y_val, val_preds, squared=False)
    weights.append(1 / fold_rmse)
    print(f'Fold {fold+1} RMSE: {fold_rmse}')
    test_preds += model.predict(test_data) * weights[fold]
test_preds /= sum(weights)

print(test_preds)

Fold 1 RMSE: 2.015768367680586
Fold 2 RMSE: 1.9103012720653367
Fold 3 RMSE: 1.9444392181946066
Fold 4 RMSE: 1.9846446917455578
Fold 5 RMSE: 2.0135768699494077
Fold 6 RMSE: 1.8855857121149355
Fold 7 RMSE: 2.2178263978508044
Fold 8 RMSE: 1.9774893491846948
Fold 9 RMSE: 2.0290644530096693
Fold 10 RMSE: 1.9689342279799642
Fold 11 RMSE: 1.9487573510703615
Fold 12 RMSE: 1.9160928425089365
Fold 13 RMSE: 2.0348872517270205
Fold 14 RMSE: 2.0091485270656766
Fold 15 RMSE: 1.9171111264336769
[ -34.90182375 -132.51930727  255.83522237 ... -627.00743922  280.45314975
  -98.89339966]
